# Recurrent Neural Networks (RNN): Advanced Tutorial

**Recurrent Neural Networks (RNNs)** are a class of neural networks designed for sequential data.
They are widely used in time series forecasting, natural language processing, and sequence modeling.

## 1. Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(42)


## 2. Generate Synthetic Sequential Data

In [ ]:
# Create a sine wave as the sequence data
timesteps = np.linspace(0, 100, 1000)
data = np.sin(timesteps)

plt.plot(timesteps, data)
plt.title("Synthetic Sine Wave")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()


## 3. Prepare Data for RNN

In [ ]:
def create_dataset(seq, input_size):
    X, y = [], []
    for i in range(len(seq) - input_size):
        X.append(seq[i:i+input_size])
        y.append(seq[i+input_size])
    return np.array(X), np.array(y)

input_size = 20
X, y = create_dataset(data, input_size)
X_tensor = torch.tensor(X).unsqueeze(-1).float()
y_tensor = torch.tensor(y).float()

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

X_tensor.shape, y_tensor.shape


## 4. Define RNN Model in PyTorch

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=1):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out.squeeze()

model = RNNModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


## 5. Train the RNN Model

In [ ]:
losses = []

for epoch in range(30):
    total_loss = 0
    for xb, yb in dataloader:
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss / len(dataloader))
    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Loss: {losses[-1]:.4f}")

plt.plot(losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.show()


## 6. Make Predictions

In [ ]:
model.eval()
with torch.no_grad():
    test_input = X_tensor[:1]
    predictions = []
    for _ in range(200):
        pred = model(test_input)
        predictions.append(pred.item())
        next_input = torch.cat((test_input[:, 1:, :], pred.unsqueeze(0).unsqueeze(2)), dim=1)
        test_input = next_input

plt.plot(data[input_size:], label="True")
plt.plot(np.arange(input_size, input_size + len(predictions)), predictions, label="Predicted")
plt.legend()
plt.title("RNN Sequence Prediction")
plt.show()


## 7. Summary

- RNNs process sequential data using hidden states
- Suitable for time series forecasting and NLP
- PyTorch provides flexible API for RNN implementation
- For long sequences, consider LSTM or GRU models